In [7]:
import pandas as pd
from sklearn.cluster import KMeans
import json

In [8]:
file_path = 'bandgap_y_sparse_cluster_50_clusters.json'
n_clusters = 50

# Read the data from the CSV file
df_data = pd.read_csv("bandgap_features.csv")

excluded = ["formula", "composition", "formula", "gap expt"]
y = df_data['gap expt'].values
X = df_data.drop(excluded, axis=1)

In [9]:
y.shape

(4604,)

In [10]:
import numpy as np
from scipy.stats import gaussian_kde

# Generate a sample one-dimensional vector
#np.random.seed(42)
#vector = np.random.randn(100)

# Estimate the density using kernel density estimation
kde = gaussian_kde(y)

# Evaluate the estimated density at sample points
sample_points = np.linspace(y.min(), y.max(), 100)
densities = kde(sample_points)

# Print the estimated density values
# for i in range(len(sample_points)):
#     print(f"Density at sample point {sample_points[i]}: {density[i]}")

In [11]:

density_dict = {}

# Print the density values for each sample position
for i in range(len(densities)):
    density = densities[i]
    #print(f"Density at sample position {i+1}: {density}")
    
    density_dict[i] = density

#print(density_dict)

sorted_density = sorted(density_dict.items(), key=lambda x: x[1])
#print(sorted_density)
top500 = [x[0] for x in sorted_density[:500]]
X_sparse = X.iloc[top500]
print(X_sparse.shape)

(100, 133)


In [17]:
id_map = {}

for i, index in enumerate(top500):
    id_map[i] = top500[i]
    

    

In [18]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, max_iter=500, random_state=222)
kmeans.fit(X_sparse)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# Print cluster labels and centers
#print("Cluster Labels:", labels)
#print("Cluster Centers:", centers)

# Create a dictionary to store the cluster assignments
cluster = {}
#print(labels)
for i, cluster_id in enumerate(labels):
    if cluster_id in cluster:
        #cluster[int(cluster_id)].append(i)
        continue
    else:
        cluster[int(cluster_id)] = [id_map[i]]







In [14]:
print(cluster)

{12: [75], 49: [76], 2: [74], 34: [77], 32: [73], 28: [78], 33: [99], 30: [96], 16: [98], 35: [95], 1: [72], 39: [94], 36: [79], 31: [71], 6: [93], 45: [80], 40: [69], 22: [92], 18: [91], 42: [90], 38: [83], 5: [66], 23: [84], 13: [86], 8: [61], 11: [59], 26: [45], 20: [44], 0: [42], 9: [41], 7: [40], 27: [39], 48: [38], 41: [34], 25: [33], 44: [31], 37: [30], 17: [27], 4: [25], 19: [23], 43: [8], 29: [9], 14: [22], 46: [21], 21: [13], 10: [14], 24: [20], 15: [4], 3: [1], 47: [0]}


In [15]:
selected_samples = []

# Print the size of each cluster
for key in sorted(cluster):
    #print(key, (cluster[key]))
    anchor_index = cluster[key][0]

    anchor_feature = X.iloc[anchor_index]
    #print(anchor_feature)
    distance_dict = {}
    for row_index in range(X.shape[0]):
        f = X.iloc[row_index]
        euclidean_distance = np.linalg.norm(anchor_feature - f)
        #print(euclidean_distance)
        
        if row_index == anchor_index:
            continue
        else:
            distance_dict[row_index] = euclidean_distance
    distance_dict = sorted(distance_dict.items(), key=lambda x: x[1])
    #print(distance_dict)
    n_added = 0
    for ck in distance_dict:
        if ck[0] not in selected_samples and n_added < 10:
            cluster[key].append(ck[0])
            selected_samples.append(ck[0])
            n_added += 1
    print(cluster[key])
    #break
for key in cluster:
    print(key, cluster[key])

[42, 10, 136, 19, 36, 5, 95, 116, 272, 85, 15]
[72, 94, 90, 87, 75, 88, 84, 92, 118, 370, 80]
[74, 97, 24, 7, 105, 96, 6, 226, 235, 31, 3]
[1, 33, 173, 519, 518, 123, 293, 187, 200, 404, 251]
[25, 121, 350, 344, 103, 86, 369, 520, 565, 321, 591]
[66, 63, 62, 55, 56, 50, 16, 20, 17, 18, 21]
[93, 73, 89, 76, 81, 115, 160, 119, 585, 109, 157]
[40, 11, 108, 170, 102, 267, 190, 222, 362, 410, 411]
[61, 57, 65, 54, 60, 64, 67, 52, 53, 48, 68]
[41, 396, 408, 107, 114, 110, 535, 315, 147, 768, 163]
[14, 175, 271, 265, 300, 230, 241, 284, 338, 264, 221]
[59, 58, 51, 49, 46, 112, 255, 172, 101, 83, 69]
[75, 72, 161, 162, 521, 399, 149, 529, 429, 442, 809]
[86, 79, 25, 256, 428, 343, 531, 326, 595, 563, 99]
[22, 12, 37, 35, 104, 34, 188, 232, 252, 185, 42]
[4, 562, 159, 93, 561, 441, 129, 253, 373, 261, 372]
[98, 43, 106, 59, 186, 356, 522, 406, 419, 227, 417]
[27, 26, 28, 30, 2, 38, 29, 201, 8, 302, 225]
[91, 781, 1, 189, 545, 228, 1151, 462, 833, 1128, 497]
[23, 176, 287, 333, 281, 135, 572, 66

In [16]:
# Example: Print formulas in cluster 0
for index in cluster[0]:
    print(df_data.iloc[index]["formula"])
    
# Save the dictionary to a file in JSON format
with open(file_path, 'w') as file:
    json.dump(cluster, file)

AgSO4
Ag2P2PbO7
AuBrO2
Ag7NO11
AgGeO3
Ag2BiO3
AlAgO2
As2O3
Ba3Eu(PO4)3
Al4Cu2O7
Ag5IO6
